<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data" data-toc-modified-id="Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data</a></span></li><li><span><a href="#Modeling" data-toc-modified-id="Modeling-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Modeling</a></span><ul class="toc-item"><li><span><a href="#test-predict" data-toc-modified-id="test-predict-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>test predict</a></span></li></ul></li></ul></div>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
plt.style.use('ggplot')
import gc
import os
import datetime
from tqdm import tqdm_notebook
import re
import xgboost as xgb
from sklearn.model_selection import KFold

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Data

In [3]:
train1 = pd.read_excel('/content/drive/My Drive/NS shop +/2020 빅콘테스트 데이터분석분야-챔피언리그_2019년 실적데이터_v1_200818.xlsx', header=1)
train2 = pd.read_excel('/content/drive/My Drive/NS shop +/2020 빅콘테스트 데이터분석분야-챔피언리그_시청률 데이터.xlsx', header=1)
test = pd.read_excel('/content/drive/My Drive/NS shop +/2020 빅콘테스트 데이터분석분야-챔피언리그_2020년 6월 판매실적예측데이터(평가데이터).xlsx', header=1)

In [4]:
train1['주문량'] = train1['취급액'] / train1['판매단가']
# Y값으로 놓을 주문량 변수 생성

In [5]:
idx = train1[train1['노출(분)'].isna()].index
for i in tqdm_notebook(idx):
    train1.loc[i, '노출(분)'] = train1.loc[i-1, '노출(분)']
train1.head()
# tqdm_notebook은 빼도 무방
# 노출(분) 컬럼의 결측값 처리

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,주문량
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,52.606516
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,109.548872
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,81.754386
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,174.310777
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,167.218045


In [6]:
idx2 = test[test['노출(분)'].isna()].index
for i in tqdm_notebook(idx2):
    test.loc[i, '노출(분)'] = test.loc[i-1, '노출(분)']
test.head()

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
0,2020-06-01 06:20:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN
1,2020-06-01 06:40:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN
2,2020-06-01 07:00:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN
3,2020-06-01 07:20:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN
4,2020-06-01 07:40:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN


In [7]:
train1.isna().sum()

방송일시        0
노출(분)       0
마더코드        0
상품코드        0
상품명         0
상품군         0
판매단가        0
취급액      2930
주문량      2930
dtype: int64

In [8]:
train1

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,주문량
0,2019-01-01 06:00:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,52.606516
1,2019-01-01 06:00:00,20.000000,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,109.548872
2,2019-01-01 06:20:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,81.754386
3,2019-01-01 06:20:00,20.000000,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,174.310777
4,2019-01-01 06:40:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,167.218045
...,...,...,...,...,...,...,...,...,...
38304,2020-01-01 00:20:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,NaN
38305,2020-01-01 00:40:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,NaN
38306,2020-01-01 01:00:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,NaN
38307,2020-01-01 01:20:00,20.000000,100490,201478,더케이 예다함 상조서비스(티포트),무형,0,NaN,NaN


In [9]:
test.isna().sum()

방송일시        0
노출(분)       0
마더코드        0
상품코드        0
상품명         0
상품군         0
판매단가        0
취급액      2891
dtype: int64

In [16]:
train1['month'] = train1['방송일시'].dt.month
train1['day'] = train1['방송일시'].dt.day
train1['hour'] = train1['방송일시'].dt.hour
train1['minute'] = train1['방송일시'].dt.minute
train1['weekday'] = train1['방송일시'].dt.weekday
test['month'] = test['방송일시'].dt.month
test['day'] = test['방송일시'].dt.day
test['hour'] = test['방송일시'].dt.hour
test['minute'] = test['방송일시'].dt.minute
test['weekday'] = test['방송일시'].dt.weekday

In [17]:
holiday_2019 = [datetime.date(2019, 1, 1), datetime.date(2019, 2, 4), datetime.date(2019, 2, 5), datetime.date(2019, 2, 6),
                datetime.date(2019, 3, 1), datetime.date(2019, 5, 5), datetime.date(2019, 5, 6), datetime.date(2019, 5, 12),
                datetime.date(2019, 4, 10), datetime.date(2019, 6, 6), datetime.date(2019, 8, 15), datetime.date(2019, 9, 12),
                datetime.date(2019, 9, 13), datetime.date(2019, 9 ,14), datetime.date(2019, 10, 3), datetime.date(2019, 10, 9),
                datetime.date(2019, 12, 25)]
holiday_2020 = [datetime.date(2020, 6, 6)]

In [18]:
def sea(x):
    if (x < 3) | (x == 12):
        return 3
    elif (x >= 3) & (x < 6):
        return 0
    elif (x >= 6) & (x < 9):
        return 1
    else:
        return 2

In [19]:
train1['season'] = train1['month'].map(sea)
test['season'] = test['month'].map(sea)
train1['holiday'] = train1['방송일시'].dt.date.map(lambda x: 1 if x in holiday_2019 else 0)
test['holiday'] = test['방송일시'].dt.date.map(lambda x: 1 if x in holiday_2020 else 0)

In [20]:
train1.head()

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,주문량,month,day,hour,minute,weekday,season,holiday
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,52.606516,1,1,6,0,1,3,1
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,109.548872,1,1,6,0,1,3,1
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,81.754386,1,1,6,20,1,3,1
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,174.310777,1,1,6,20,1,3,1
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,167.218045,1,1,6,40,1,3,1


In [21]:
test.head()

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,month,day,hour,minute,weekday,season,holiday
0,2020-06-01 06:20:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,6,1,6,20,0,1,0
1,2020-06-01 06:40:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,6,1,6,40,0,1,0
2,2020-06-01 07:00:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,6,1,7,0,0,1,0
3,2020-06-01 07:20:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN,6,1,7,20,0,1,0
4,2020-06-01 07:40:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN,6,1,7,40,0,1,0


# Modeling

- 검증데이터는 2019년 6월꺼로 (일단)
- 데이터가 적어 kfold 사용
- 일단 주어진 컬럼들만 사용해서 예측해 봄
- 무형은 빼고, 취급액 NaN도 빼고 학습
- test set 예측 시 상품군이 무형인 것들은 취급액 NaN으로 넣음
- 모델은 일단 xgboost로 해보았음
- 주문량을 예측해서 판매단가에 곱한 것을 취급액 예측치로

In [26]:
cols = ['노출(분)', '마더코드', '상품코드', '상품군', '판매단가', 'day', 'hour', 'minute', 'weekday', 'season', 'holiday']
x_train = train1[(train1['month'] != 6) & (train1['상품군'] != '무형') & (train1['주문량'].isna() != True)][cols]
y_train = train1[(train1['month'] != 6) & (train1['상품군'] != '무형') & (train1['주문량'].isna() != True)][['주문량']]
x_valid = train1[(train1['month'] == 6) & (train1['상품군'] != '무형') & (train1['주문량'].isna() != True)][cols]
y_valid = train1[(train1['month'] == 6) & (train1['상품군'] != '무형') & (train1['주문량'].isna() != True)][['주문량']]
x_test = test[cols]
real_y_train = train1[(train1['month'] != 6) & (train1['상품군'] != '무형') & (train1['주문량'].isna() != True)][['취급액']]
real_y_valid = train1[(train1['month'] == 6) & (train1['상품군'] != '무형') & (train1['주문량'].isna() != True)][['취급액']]
x_train.head()

,노출(분),마더코드,상품코드,상품군,판매단가,day,hour,minute,weekday,season,holiday
0,20.0,100346,201072,의류,39900,1,6,0,1,3,1
1,20.0,100346,201079,의류,39900,1,6,0,1,3,1
2,20.0,100346,201072,의류,39900,1,6,20,1,3,1
3,20.0,100346,201079,의류,39900,1,6,20,1,3,1
4,20.0,100346,201072,의류,39900,1,6,40,1,3,1


In [27]:
x_valid.head()

,노출(분),마더코드,상품코드,상품군,판매단가,day,hour,minute,weekday,season,holiday
16109,20.0,100380,201243,의류,39900,1,1,0,5,1,0
16110,20.0,100380,201245,의류,39900,1,1,0,5,1,0
16111,20.0,100380,201243,의류,39900,1,1,20,5,1,0
16112,20.0,100380,201245,의류,39900,1,1,20,5,1,0
16113,13.0,100380,201243,의류,39900,1,1,40,5,1,0


In [28]:
x_test.head()

,노출(분),마더코드,상품코드,상품군,판매단가,day,hour,minute,weekday,season,holiday
0,20.0,100650,201971,의류,59800,1,6,20,0,1,0
1,20.0,100650,201971,의류,59800,1,6,40,0,1,0
2,20.0,100650,201971,의류,59800,1,7,0,0,1,0
3,20.0,100445,202278,속옷,69900,1,7,20,0,1,0
4,20.0,100445,202278,속옷,69900,1,7,40,0,1,0


In [29]:
def a(x):
    d1 = {'의류': 1, '속옷': 2, '주방': 3, '농수축': 4, '이미용': 5, '가전': 6, '생활용품': 7,
          '건강기능': 8, '잡화': 9, '가구': 10, '침구': 11, '무형': 12}
    return d1[x]
# 상품군 그냥 숫자로

In [30]:
x_train['상품군'] = x_train['상품군'].map(a)
x_valid['상품군'] = x_valid['상품군'].map(a)
#x_test['상품군'] = x_test['상품군'].map(a)
x_train.head()

,노출(분),마더코드,상품코드,상품군,판매단가,day,hour,minute,weekday,season,holiday
0,20.0,100346,201072,1,39900,1,6,0,1,3,1
1,20.0,100346,201079,1,39900,1,6,0,1,3,1
2,20.0,100346,201072,1,39900,1,6,20,1,3,1
3,20.0,100346,201079,1,39900,1,6,20,1,3,1
4,20.0,100346,201072,1,39900,1,6,40,1,3,1


In [31]:
x_valid.head()

,노출(분),마더코드,상품코드,상품군,판매단가,day,hour,minute,weekday,season,holiday
16109,20.0,100380,201243,1,39900,1,1,0,5,1,0
16110,20.0,100380,201245,1,39900,1,1,0,5,1,0
16111,20.0,100380,201243,1,39900,1,1,20,5,1,0
16112,20.0,100380,201245,1,39900,1,1,20,5,1,0
16113,13.0,100380,201243,1,39900,1,1,40,5,1,0


In [32]:
x_test.head()

,노출(분),마더코드,상품코드,상품군,판매단가,day,hour,minute,weekday,season,holiday
0,20.0,100650,201971,의류,59800,1,6,20,0,1,0
1,20.0,100650,201971,의류,59800,1,6,40,0,1,0
2,20.0,100650,201971,의류,59800,1,7,0,0,1,0
3,20.0,100445,202278,속옷,69900,1,7,20,0,1,0
4,20.0,100445,202278,속옷,69900,1,7,40,0,1,0


In [33]:
xgb_params={'eta':0.05,
            'max_depth':6,
            'objective':'reg:squarederror',
            'eval_metric':'mae',
            'subsample':0.9,
            'colsample_bytree':0.9,
            'min_child_weight':1,
            'seed':34}
# 모델의 파라미터들
# 내 맘대로 일단 넣어봄

In [34]:
def xgb_mape(preds, dtrain):
    labels = dtrain.get_label()
    return('mape', np.mean(np.abs((labels - preds) / (labels))) * 100)

In [35]:
%%time

n_ = 5
kf = KFold(n_splits=n_, shuffle=True, random_state=42)
predictions = np.zeros(len(x_valid))

for fold_, (trn_idx, val_idx) in enumerate(kf.split(x_train, y_train)):
    print("fold num_: {}".format(fold_))
    trn_data = xgb.DMatrix(x_train.iloc[trn_idx], label=y_train.iloc[trn_idx])
    val_data = xgb.DMatrix(x_train.iloc[val_idx], label=y_train.iloc[val_idx])

    watchlist = [(trn_data, 'train'), (val_data, 'valid')]
    num_round = 5000
    mod = xgb.train(params = xgb_params,
                    dtrain = trn_data,
                    num_boost_round = num_round,
                    evals = watchlist,
                    feval = xgb_mape,
                    verbose_eval = 50,
                    early_stopping_rounds = 50)

    predictions += mod.predict(xgb.DMatrix(x_valid), ntree_limit=mod.best_iteration) / n_
    
print('\nCross Validation Is Complete')

fold num_: 0
[0]	train-mae:299.258	valid-mae:300.52	train-mape:88.4774	valid-mape:88.139
Multiple eval metrics have been passed: 'valid-mape' will be used for early stopping.

Will train until valid-mape hasn't improved in 50 rounds.
[50]	train-mae:112.054	valid-mae:114.915	train-mape:118.674	valid-mape:117.918
Stopping. Best iteration:
[4]	train-mae:246.993	valid-mae:248.069	train-mape:71.5624	valid-mape:70.252

fold num_: 1
[0]	train-mae:299.043	valid-mae:301.537	train-mape:88.4583	valid-mape:88.6215
Multiple eval metrics have been passed: 'valid-mape' will be used for early stopping.

Will train until valid-mape hasn't improved in 50 rounds.
[50]	train-mae:111.154	valid-mae:117.537	train-mape:118.022	valid-mape:120.242
Stopping. Best iteration:
[4]	train-mae:246.729	valid-mae:249.729	train-mape:70.9342	valid-mape:71.8531

fold num_: 2
[0]	train-mae:300.588	valid-mae:295.145	train-mape:88.3098	valid-mape:88.6223
Multiple eval metrics have been passed: 'valid-mape' will be used for ea

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
predictions[:20]

array([117.76170158, 117.76170158, 117.76170158, 117.76170158,
       128.94736099, 128.94736099,  64.13750648,  70.32392502,
        90.92757225,  86.05611229,  94.91324139, 166.17050171,
       148.83016396, 156.51192284, 250.22752762,  16.66899014,
        16.66899014,  31.15714979,  31.15714979,  37.89097691])

In [37]:
def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [38]:
mape(y_valid, predictions)

328.44359731624564

In [39]:
y_pred = x_valid['판매단가'] * predictions
y_pred = y_pred.round(-3)
y_pred.head()

16109    4699000.0
16110    4699000.0
16111    4699000.0
16112    4699000.0
16113    5145000.0
Name: 판매단가, dtype: float64

In [40]:
mape(real_y_valid, y_pred)
# 검증데이터 mape

74.71115991331261

In [45]:
from sklearn.metrics import accuracy_score, confusion_matrix, r2_score
r2_score(y_valid, predictions)

-0.1883800947392391

## test predict

In [ ]:
x_train = train1[(train1['상품군'] != '무형') & (train1['주문량'].isna() != True)][cols]
y_train = train1[(train1['상품군'] != '무형') & (train1['주문량'].isna() != True)][['주문량']]
x_train['상품군'] = x_train['상품군'].map(a)
x_train.head()

,노출(분),마더코드,상품코드,상품군,판매단가,day,hour,minute,weekday,season,holiday
0,20.0,100346,201072,1,39900,1,6,0,1,3,1
1,20.0,100346,201079,1,39900,1,6,0,1,3,1
2,20.0,100346,201072,1,39900,1,6,20,1,3,1
3,20.0,100346,201079,1,39900,1,6,20,1,3,1
4,20.0,100346,201072,1,39900,1,6,40,1,3,1


In [ ]:
%%time

n_ = 5
kf = KFold(n_splits=n_, shuffle=True, random_state=42)
xgb_pred = np.zeros(len(x_test))

for fold_, (trn_idx, val_idx) in enumerate(kf.split(x_train, y_train)):
    print("fold num_: {}".format(fold_))
    trn_data = xgb.DMatrix(x_train.iloc[trn_idx], label=y_train.iloc[trn_idx])
    val_data = xgb.DMatrix(x_train.iloc[val_idx], label=y_train.iloc[val_idx])

    watchlist = [(trn_data, 'train'), (val_data, 'valid')]
    num_round = 5000
    mod = xgb.train(params = xgb_params,
                    dtrain = trn_data,
                    num_boost_round = num_round,
                    evals = watchlist,
                    feval = xgb_mape,
                    verbose_eval = 50,
                    early_stopping_rounds = 50)

    xgb_pred += mod.predict(xgb.DMatrix(x_test), ntree_limit=mod.best_iteration) / n_
    
print('\nCross Validation Is Complete')

fold num_: 0
[0]	train-mae:298.408	valid-mae:300.132	train-mape:88.3763	valid-mape:88.2383
Multiple eval metrics have been passed: 'valid-mape' will be used for early stopping.

Will train until valid-mape hasn't improved in 50 rounds.
[50]	train-mae:109.99	valid-mae:113.838	train-mape:113.033	valid-mape:116.276
Stopping. Best iteration:
[7]	train-mae:216.43	valid-mae:217.856	train-mape:68.5412	valid-mape:69.0524

fold num_: 1
[0]	train-mae:299.194	valid-mae:296.686	train-mape:88.4616	valid-mape:88.3493
Multiple eval metrics have been passed: 'valid-mape' will be used for early stopping.

Will train until valid-mape hasn't improved in 50 rounds.
[50]	train-mae:110.188	valid-mae:112.812	train-mape:113.957	valid-mape:113.012
Stopping. Best iteration:
[7]	train-mae:216.849	valid-mae:214.393	train-mape:68.8118	valid-mape:68.3688

fold num_: 2
[0]	train-mae:300.395	valid-mae:292.243	train-mape:88.4338	valid-mape:88.3979
Multiple eval metrics have been passed: 'valid-mape' will be used for e

In [ ]:
xgb_pred[:20]

array([ 73.8034277 , 103.93289375, 104.45034599,  89.51972961,
       118.70150566,  89.8853054 , 140.39179611, 179.45199585,
       190.70992661,  73.94635105, 107.15302658,  70.04640961,
       130.41251755, 173.18014908, 130.41251755, 291.8818779 ,
       355.59360504, 247.4059639 ,  76.46537685, 126.03474998])

In [ ]:
test_pred = x_test['판매단가'] * xgb_pred
test_pred = test_pred.round(-3)
test_pred.head()

0    4413000.0
1    6215000.0
2    6246000.0
3    6257000.0
4    8297000.0
Name: 판매단가, dtype: float64

In [ ]:
test_set = pd.read_excel('02_평가데이터/2020 빅콘테스트 데이터분석분야-챔피언리그_2020년 6월 판매실적예측데이터(평가데이터).xlsx', header=1)
test_set['취급액'] = test_pred
test_set.head()

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
0,2020-06-01 06:20:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,4413000.0
1,2020-06-01 06:40:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,6215000.0
2,2020-06-01 07:00:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,6246000.0
3,2020-06-01 07:20:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,6257000.0
4,2020-06-01 07:40:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,8297000.0


In [ ]:
test_set['취급액'][test_set['상품군'] == '무형'] = np.NaN
test_set.head()
# 최종 submission 형태

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
0,2020-06-01 06:20:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,4413000.0
1,2020-06-01 06:40:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,6215000.0
2,2020-06-01 07:00:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,6246000.0
3,2020-06-01 07:20:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,6257000.0
4,2020-06-01 07:40:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,8297000.0


In [ ]:
test_set.isna().sum()

방송일시        0
노출(분)    1111
마더코드        0
상품코드        0
상품명         0
상품군         0
판매단가        0
취급액       175
dtype: int64

In [ ]:
# test_set.to_csv('base_submission_xgb2.csv', index=False, encoding='utf-8-sig')

In [ ]:
# import joblib
# joblib.dump(mod, 'base_model_xgb.model')

In [ ]:
# model = joblib.load('base_model_xgb.model')
# model load